### data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os

### Set

In [2]:
# generation_q = pd.read_csv(
#     "/Users/jangseohyun/Documents/workspace/symply/DER/DATA_generation.csv"
# )
# generation_q["Time"] = pd.to_datetime(generation_q["Time"], format="%Y-%m-%d %H:%M")
# generation_q["Hour"] = generation_q["Time"].dt.floor("h")
# generation_h = generation_q.groupby("Hour").sum(numeric_only=True)

# demand_q = pd.read_csv(
#     "/Users/jangseohyun/Documents/workspace/symply/DER/DATA_demand.csv"
# )
# demand_q["Time"] = pd.to_datetime(demand_q["Time"], format="%Y-%m-%d %H:%M")
# demand_q["Hour"] = demand_q["Time"].dt.floor("h")
# demand_h = demand_q.groupby("Hour").sum(numeric_only=True)

# price_q = pd.read_csv(
#     "/Users/jangseohyun/Documents/workspace/symply/DER/DATA_price.csv"
# )
# price_q["Time"] = pd.to_datetime(price_q["Time"], format="%Y-%m-%d %H:%M")

In [3]:
# I = list(range(len(generation_q.columns) - 11))
# T = list(generation_h.index.hour.unique())
# S = list(range(20))
# prob = np.array([1 / len(S) for s in S])

<!-- #### Generation -->

In [4]:
# generation_avg = np.array(
#     [[generation_h[generation_h.index.hour == t].mean().iloc[i] for t in T] for i in I]
# )


# def generate_randomized_generation(I, T, S, generation_avg, randomness_level):
#     np.random.seed(7)
#     if randomness_level == "low":
#         noise_factors = np.random.uniform(0.8, 1.2, size=(len(I), len(T), len(S)))
#     elif randomness_level == "medium":
#         noise_factors = np.random.uniform(0.5, 1.5, size=(len(I), len(T), len(S)))
#     elif randomness_level == "high":
#         noise_factors = np.random.uniform(0.2, 1.8, size=(len(I), len(T), len(S)))
#     elif randomness_level == "none":
#         noise_factors = np.random.uniform(1, 1.1, size=(len(I), len(T), len(S)))
#     else:
#         raise ValueError(
#             "Invalid randomness level. Please choose 'low', 'medium', or 'high'."
#         )

#     generation_r = np.zeros((len(I), len(T), len(S)))
#     for i in range(len(I)):
#         for t in range(len(T)):
#             for s in range(len(S)):
#                 generation_r[i, t, s] = generation_avg[i, t] * noise_factors[i, t, s]

#     return generation_r

<!-- #### Demand -->

In [5]:
# demand_avg = np.array(
#     [[demand_h[demand_h.index.hour == t].mean().iloc[i] for t in T] for i in I]
# )


# def generate_randomized_demand(I, T, S, demand_avg, randomness_level):
#     np.random.seed(17)
#     if randomness_level == "low":
#         noise_factors = np.random.uniform(0.8, 1.2, size=(len(I), len(T), len(S)))
#     elif randomness_level == "medium":
#         noise_factors = np.random.uniform(0.5, 1.5, size=(len(I), len(T), len(S)))
#     elif randomness_level == "high":
#         noise_factors = np.random.uniform(0.2, 1.8, size=(len(I), len(T), len(S)))
#     elif randomness_level == "none":
#         noise_factors = np.random.uniform(1, 1.1, size=(len(I), len(T), len(S)))
#     else:
#         raise ValueError(
#             "Invalid randomness level. Please choose 'low', 'medium', or 'high'."
#         )

#     demand_r = np.zeros((len(I), len(T), len(S)))
#     for i in range(len(I)):
#         for t in range(len(T)):
#             for s in range(len(S)):
#                 demand_r[i, t, s] = demand_avg[i, t] * noise_factors[i, t, s]

#     return demand_r

<!-- #### Randomize -->

In [6]:
# random_key = "high"

In [7]:
# generation_r = generate_randomized_generation(I, T, S, generation_avg, random_key)
# demand_r = generate_randomized_demand(I, T, S, demand_avg, random_key)

<!-- #### Residual = generation - demand -->

In [8]:
# residual = np.zeros((len(I), len(T), len(S)))
# for i in range(len(I)):
#     for t in range(len(T)):
#         for s in range(len(S)):
#             residual[i, t, s] = generation_r[i, t, s] - demand_r[i, t, s]

# R = np.zeros((len(I), len(T), len(S)))
# for i in range(len(I)):
#     for t in range(len(T)):
#         for s in range(len(S)):
#             R[i, t, s] = max(0, residual[i, t, s])

In [9]:
# index = pd.MultiIndex.from_product([range(len(I)), range(len(T)), range(len(S))],
#                                  names=['generator', 'time', 'scenario'])
# df = pd.DataFrame({'value': R.flatten()}, index=index).reset_index()
# df.to_csv('result/result_R.csv', index=False)

<!-- #### Price 설정 -->

In [10]:
# price_q["Hour"] = price_q["Time"].dt.floor("h")
# price_h = price_q.groupby("Hour").mean(numeric_only=True)

# price = price_h.iloc[: len(S) * len(T)]

# P_DA = np.array(
#     [sum(price["Price"].iloc[t + s * len(T)] for s in S) / len(S) * 1.2 for t in T]
# )
# P_RT = np.array([[price["Price"].iloc[t + s * len(T)] for s in S] for t in T])
# P_PN = np.array(
#     [sum(price["Price"].iloc[t + s * len(T)] for s in S) / len(S) * 2 for t in T]
# )

### 모델 DER Aggregation 

In [11]:
def load_generation_data(include_files=None, date_filter=None):
    data_dir = "/Users/jangseohyun/Documents/workspace/symply/DER/data/generation"
    all_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.csv')])

    if include_files is not None:
        for file in include_files:
            if file not in all_files:
                raise ValueError(f"파일을 찾을 수 없습니다: {file}")
        files_to_load = [f for f in all_files if f in include_files]
    else:
        files_to_load = all_files

    I = len(files_to_load)  
    T = 24 
    generation_data = np.zeros((I, T)) 

    loaded_files = []
    
    for idx, file in enumerate(files_to_load):
        file_path = os.path.join(data_dir, file)
        df = pd.read_csv(file_path)
        df.columns = df.columns.str.strip()

        date_col = "Date"
        hour_col = "Hour (Eastern Time, Daylight-Adjusted)"
        gen_col = "Electricity Generated"

        if any(col not in df.columns for col in [date_col, hour_col, gen_col]):
            print(f"{file}: 필요한 컬럼 없음. 스킵됨.")
            continue

        if date_filter:
            df = df[df[date_col] == date_filter]
            if df.empty:
                print(f"{file}: {date_filter} 데이터 없음. 스킵됨.")
                continue 

        df = df[df[hour_col].astype(str).str.match(r'^\d+$')]
        df["Time"] = df[hour_col].astype(int)
        df = df[df["Time"].between(0, 23)]

        for t in range(T):
            if t in df["Time"].values:
                generation_data[idx, t] = df[df["Time"] == t][gen_col].values[0]

        loaded_files.append(file)

    print(f"✅ 총 {I}개 파일을 불러왔습니다: {', '.join(loaded_files)}")

    return generation_data, I, T

def generate_randomized_generation(I, T, S, data, randomness_level):
    np.random.seed(7)

    noise_ranges = {
        "low": (0.8, 1.2),
        "medium": (0.5, 1.5),
        "high": (0.2, 1.8),
    }

    if randomness_level not in noise_ranges:
        raise ValueError("Invalid randomness level. Please choose 'low', 'medium', or 'high'.")

    low, high = noise_ranges[randomness_level]
    noise_factors = np.random.uniform(low, high, size=(I, T, S))

    generation_r = np.expand_dims(data, axis=-1) * noise_factors
    
    print(f"📊 데이터 Shape: I={I}, T={T}, S={S}")
    return generation_r

def plot_generation_data(generation_data, I):
    hours = np.arange(24)
    plt.figure(figsize=(15, 9))

    for i in range(I):
        plt.plot(hours, generation_data[i], marker='o', linestyle='-', alpha=0.7, label=f'Generator {i}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title("Hourly Electricity Generation for All Generators")
    plt.xticks(hours)  # 0~23 시간 설정
    plt.legend(loc="upper left", fontsize='small')

    plt.show()

def plot_randomized_generation(R, I, T, S):
    hours = np.arange(T)
    
    plt.figure(figsize=(15, 9))

    for i in range(I):
        plt.plot(hours, R[i, :, S], marker='o', linestyle='-', alpha=0.7, label=f'Generator {i}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title(f"Randomized Hourly Generation for Scenario {S}")
    plt.xticks(hours) 
    plt.legend(loc="upper left") 

    plt.show()
       
def plot_scenarios_for_generator(R, i):

    T = R.shape[1]
    S = R.shape[2] 
    hours = np.arange(T) 

    plt.figure(figsize=(15, 9))

    for s in range(S):
        plt.plot(hours, R[i, :, s], linestyle='-', alpha=0.6, label=f'Scenario {s+1}')

    plt.xlabel("Hour")
    plt.ylabel("Electricity Generated (kWh)")
    plt.title(f"Hourly Electricity Generation for Generator {i} Across All Scenarios")
    plt.xticks(hours)
    plt.legend(loc="upper left", fontsize='small', ncol=2)
    plt.show()

def generate_rt_scenarios(rt_da, S, randomness_level):

    rt_da["Time Stamp"] = pd.to_datetime(rt_da["Time Stamp"])
    nyc_rt = rt_da[rt_da["Name"] == "N.Y.C."].copy() 

    nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")
    hourly_avg = nyc_rt.groupby("Hour")["LBMP ($/MWHr)"].mean().reset_index()
    price_hourly = hourly_avg["LBMP ($/MWHr)"].to_numpy()
    T = len(price_hourly)

    noise_ranges = {
        "low": (0.95, 1.05),
        "medium": (0.85, 1.15),
        "high": (0.7, 1.3),
    }

    if randomness_level not in noise_ranges:
        raise ValueError("Invalid randomness level. Choose from 'low', 'medium', 'high'.")

    low, high = noise_ranges[randomness_level]
    noise_factors = np.random.uniform(low, high, size=(T, S))

    P_RT = np.expand_dims(price_hourly, axis=-1) * noise_factors

    return P_RT

def plot_rt_scenarios(P_RT):
    T, S = P_RT.shape
    hours = np.arange(T)

    plt.figure(figsize=(15, 8))

    for s in range(S):
        plt.plot(hours, P_RT[:, s], linestyle='-', alpha=0.6, label=f"Scenario {s+1}")

    plt.xlabel("Hour")
    plt.ylabel("Price ($/MWHr)")
    plt.title("Real-Time Price Scenarios (Hourly Averaged)")
    plt.xticks(hours)
    plt.legend(loc="upper left", fontsize="small", ncol=2)

    plt.show()
    
only_profit = np.array(pd.read_csv("result/result_only_profit.csv"))
ny_da = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718da.csv")
ny_rt = pd.read_csv("/Users/jangseohyun/Documents/workspace/symply/DER/data/price/20220718rt.csv")
ny_da["Time Stamp"] = pd.to_datetime(ny_da["Time Stamp"])
ny_da["Hour"] = ny_da["Time Stamp"].dt.hour
nyc_data = ny_da[ny_da["Name"] == "N.Y.C."]
P_DA = nyc_data["LBMP ($/MWHr)"].to_numpy() * 1.25
P_PN = P_DA * 1.5

# plot_generation_data(generation_data, 10)
# plot_randomized_generation(R, I, T, 7)
# plot_scenarios_for_generator(R, 1)
# plot_rt_scenarios(P_RT)

include_files = ['1201.csv', '137.csv', '281.csv', '397.csv', '401.csv', '430.csv', '514.csv', '524.csv', '775.csv', '89.csv']
generation_data, I, T = load_generation_data(include_files, "2022-07-18")

S = 20
randomness_level = "medium"
R = generate_randomized_generation(I, T, S, generation_data, randomness_level)
P_RT = generate_rt_scenarios(ny_rt, S, randomness_level)

✅ 총 10개 파일을 불러왔습니다: 1201.csv, 137.csv, 281.csv, 397.csv, 401.csv, 430.csv, 514.csv, 524.csv, 775.csv, 89.csv
📊 데이터 Shape: I=10, T=24, S=20


/var/folders/2_/9m9qjphn0xs45xzpvf1hl0j00000gn/T/ipykernel_60258/2950288952.py:126: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")


In [12]:
agg = gp.Model("agg")
# agg.Params.OutputFlag = 0
agg.Params.MIPGap = 0.00001  # MIP gap을 1%로 설정


alpha = agg.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
beta_plus = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
beta_minus = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")

M = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
z = agg.addVars(T, S, vtype=GRB.BINARY, name="z")
prob = np.array([1 / S for s in range(S)])

agg.update()

obj = gp.quicksum(P_DA[t] * alpha[t] for t in range(T)) + gp.quicksum(
    prob[s] * (P_RT[t, s] * beta_plus[t, s] - P_PN[t] * beta_minus[t, s])
    for t in range(T)
    for s in range(S)
)

agg.setObjective(obj, GRB.MAXIMIZE)

Set parameter Username
Set parameter LicenseID to value 2611964
Academic license - for non-commercial use only - expires 2026-01-20
Set parameter MIPGap to value 1e-05


In [13]:
for t in range(T):
    for s in range(S):
        agg.addConstr(
            gp.quicksum(R[i, t, s] for i in range(I)) - alpha[t]
            == beta_plus[t, s] - beta_minus[t, s]
        )

for t in range(T):
    for s in range(S):
        agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) >= beta_plus[t, s])

for t in range(T):
    for s in range(S):
        agg.addConstr(beta_plus[t, s] <= M * z[t, s])
        agg.addConstr(beta_minus[t, s] <= M * (1 - z[t, s]))

agg.optimize()

if agg.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Objective value: {agg.objVal}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  1e-05

Optimize a model with 1920 rows, 1464 columns and 3840 nonzeros
Model fingerprint: 0x9bb750f6
Variable types: 984 continuous, 480 integer (480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [3e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-02, 2e+03]
Found heuristic solution: objective 1589280.4391
Presolve removed 1860 rows and 1403 columns
Presolve time: 0.01s
Presolved: 60 rows, 61 columns, 140 nonzeros
Found heuristic solution: objective 2085445.7362
Variable types: 41 continuous, 20 integer (20 binary)

Root relaxation: objective 2.085545e+06, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

### 결과 분석

#### 수익 분석

In [14]:
# Day-ahead 수익 계산
total_da_profit_obj = 0
for t in range(T):
    total_da_profit_obj += P_DA[t] * alpha[t].x  

# Real-time 수익 계산
total_rt_profit_obj = 0
for t in range(T):
    for s in range(S):
        rt_profit_obj = P_RT[t, s] * beta_plus[t, s].x  
        total_rt_profit_obj += prob[s] * rt_profit_obj

# Penalty 비용 계산
total_penalty_cost_obj = 0
for t in range(T):
    for s in range(S):
        penalty_cost_obj = P_PN[t] * beta_minus[t, s].x
        total_penalty_cost_obj += prob[s] * penalty_cost_obj

# 총 시스템 이익 (목적 함수 기반)
total_system_profit_obj = total_da_profit_obj + total_rt_profit_obj - total_penalty_cost_obj

# 결과 출력
print(f"DA: {total_da_profit_obj:.2f}")
print(f"RT: {total_rt_profit_obj:.2f}")
print(f"Penalty: {total_penalty_cost_obj:.2f}")
print(f"목적 함수 기반 총 이익: {total_system_profit_obj:.2f}")

DA: 568659.67
RT: 1531172.09
Penalty: 14287.13
목적 함수 기반 총 이익: 2085544.63


#### 하루 aggregated 커밋량 분석

In [15]:
# 각 시간대 t에 대한 평균 alpha, beta+ 및 beta- 계산 및 출력
for t in range(T):
    avg_alpha = alpha[t].x  # alpha[t]의 최적화 값
    avg_beta_plus = sum(beta_plus[t, s].x for s in range(S)) / S  # beta_plus의 평균
    avg_beta_minus = sum(beta_minus[t, s].x for s in range(S)) / S  # beta_minus의 평균

    print(f"[시간 {t}] alpha: {avg_alpha:.3f}, beta+: {avg_beta_plus:.3f}, beta-: {avg_beta_minus:.3f}")

# alpha 값의 총합 출력
total_alpha = sum(alpha[t].x for t in range(T))
print(f"총 하루 commitment: {total_alpha:.3f}")

[시간 0] alpha: 0.000, beta+: 0.000, beta-: 0.000
[시간 1] alpha: 0.000, beta+: 0.127, beta-: 0.000
[시간 2] alpha: 0.132, beta+: 0.098, beta-: 0.000
[시간 3] alpha: 0.000, beta+: 0.000, beta-: 0.000
[시간 4] alpha: 0.000, beta+: 0.760, beta-: 0.000
[시간 5] alpha: 0.000, beta+: 0.826, beta-: 0.000
[시간 6] alpha: 7.703, beta+: 0.841, beta-: 0.161
[시간 7] alpha: 0.000, beta+: 41.808, beta-: 0.000
[시간 8] alpha: 81.988, beta+: 25.069, beta-: 0.382
[시간 9] alpha: 261.561, beta+: 24.095, beta-: 8.507
[시간 10] alpha: 626.286, beta+: 80.442, beta-: 1.969
[시간 11] alpha: 0.000, beta+: 893.379, beta-: 0.000
[시간 12] alpha: 0.000, beta+: 1263.720, beta-: 0.000
[시간 13] alpha: 0.000, beta+: 1766.632, beta-: 0.000
[시간 14] alpha: 1655.569, beta+: 169.851, beta-: 40.808
[시간 15] alpha: 0.000, beta+: 1178.700, beta-: 0.000
[시간 16] alpha: 786.512, beta+: 151.178, beta-: 3.201
[시간 17] alpha: 0.000, beta+: 920.147, beta-: 0.000
[시간 18] alpha: 0.000, beta+: 692.545, beta-: 0.000
[시간 19] alpha: 391.276, beta+: 54.208, beta-:

### 사후정산

In [16]:
only_value = pd.read_csv('result/result_only_obj.csv').values
only_profit = pd.read_csv('result/result_only_profit.csv').values
surplus = agg.objVal - only_value[0] #

I, T, S = R.shape

R_proportion = R / R.sum(axis=1, keepdims=True) 
R_proportion = np.nan_to_num(R_proportion) 

R_proportion_P = np.multiply(R_proportion, P_DA[:, np.newaxis])

R_weighted = R_proportion_P.sum(axis=(1, 2)) 

R_weighted_normalized = R_weighted / R_weighted.sum() 

surplus_distribution = surplus * R_weighted_normalized  

final_profit = only_profit.flatten() + surplus_distribution

print("Surplus 분배 결과:")
for i, value in enumerate(surplus_distribution):
    print(f"[{i}]: {value:.2f}")

print("\n최종 Profit:")
for i, (profit, only) in enumerate(zip(final_profit, only_profit.flatten())):
    increase_percentage = ((profit - only) / only) * 100
    print(f"[{i}] {profit:.2f} ({increase_percentage:.2f}%)")

Surplus 분배 결과:
[0]: 103.56
[1]: 106.27
[2]: 104.10
[3]: 101.63
[4]: 103.95
[5]: 104.58
[6]: 107.17
[7]: 108.07
[8]: 99.89
[9]: 105.14

최종 Profit:
[0] 202682.79 (0.05%)
[1] 338681.08 (0.03%)
[2] 190161.64 (0.05%)
[3] 52219.55 (0.19%)
[4] 373182.00 (0.03%)
[5] 93251.68 (0.11%)
[6] 91194.97 (0.12%)
[7] 456050.12 (0.02%)
[8] 130766.97 (0.08%)
[9] 157353.85 (0.07%)


### 결과 저장

In [17]:
alpha_df = pd.DataFrame({
    'alpha': [alpha[t].x for t in range(T)]
})
alpha_df.to_csv('result/result_base_alpha.csv', index=False)

# # beta_plus 저장
# beta_plus_df = pd.DataFrame(
#     [[beta_plus[t,s].x for s in range(S)] 
#      for t in range(T)],
#     columns=[f'S{s}' for s in range(S)]
# )
# beta_plus_df.to_csv('result_beta_plus.csv', index=False)

# # beta_minus 저장
# beta_minus_df = pd.DataFrame(
#     [[beta_minus[t,s].x for s in range(S)] 
#      for t in range(T)],
#     columns=[f'S{s}' for s in range(S)]
# )
# beta_minus_df.to_csv('result_beta_minus.csv', index=False)

In [18]:
agg_profit_value = pd.DataFrame({'agg_profit_value': final_profit})
agg_profit_value.to_csv('result/result_agg_profit.csv', index=False)

agg_obj = pd.DataFrame({'agg_obj': [agg.objVal]})
agg_obj.to_csv('result/result_agg_obj.csv', index=False)